In [ ]:
#Load datasets
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

batch_size = 500

trainset_MNIST_big = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainset_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000)))
trainloader_MNIST = torch.utils.data.DataLoader(trainset_MNIST, batch_size=batch_size, shuffle=True, num_workers=2)

val_set_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000, 20000)))
val_loader_MNIST = torch.utils.data.DataLoader(val_set_MNIST, batch_size=batch_size, shuffle=True, num_workers=2)
                                          
testset_MNIST = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader_MNIST = torch.utils.data.DataLoader(testset_MNIST, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# define model
model = models.resnet18(pretrained= False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_model = model
    best_loss = 100
    for epoch in range(num_epochs):
        tr_correct = 0
        tr_total = 0
        val_correct = 0
        val_total = 0
        for batch_nr, (data, labels) in enumerate(train_loader):
            
            print("Epoch: ",epoch,"Batch: ",batch_nr)
            # calculate prediction according to our model
            
            prediction = model.forward(data)
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)
            
            # Backpropagate the loss through the network to find the gradients of all parameters
            loss.backward()
            
            # Update the parameters along their gradients
            optimizer.step()
            
            # Clear stored gradient values
            optimizer.zero_grad()

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    tr_correct+=1
                tr_total +=1

        for batch_nr, (data, labels) in enumerate(val_loader):
            
            prediction = model.forward(data)
            
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)

            if(loss < best_loss):
                best_loss = loss
                best_model = model

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    val_correct+=1
                val_total +=1

    # primt accuracy
    tr_accuracy = tr_correct/tr_total
    val_accuracy = val_correct/val_total
    print(f'Training accuracy:   {str(100*tr_accuracy)[:4]}%.')
    print(f'Validation accuracy: {str(100*val_accuracy)[:4]}%.')

    return best_model

In [ ]:
def test(model, test_loader):
    val_correct = 0
    val_total = 0
    for batch_nr, (data, labels) in enumerate(test_loader):
        prediction = model.forward(data)
        
        # calculate accuracy
        for i in range(len(data)):    
            guess = torch.argmax(prediction[i])
            if(guess.item() == labels[i]):
                val_correct+=1
            val_total +=1

    # primt accuracy
    val_accuracy = val_correct/val_total
    print(f'Test accuracy: {str(100*val_accuracy)[:4]}%.')

In [ ]:
Trained_model = train(model, criterion, optimizer, trainloader_MNIST, val_loader_MNIST, num_epochs=10)
test(Trained_model, testloader_MNIST)

In [ ]:


trained_resnet_fe = Trained_model 

# FREEZE all old params
for param in trained_resnet_fe.parameters():
    param.requires_grad = False

# new layer
num_ftrs = trained_resnet_fe.fc.in_features
trained_resnet_fe.fc = nn.Linear(num_ftrs, 10)

# find the paramaters we want to update during training
params_to_update = []
for param in trained_resnet_fe.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)